# By-Band _g_-Point Reduction

In [ ]:
%load_ext autoreload
%autoreload 2

# Dependencies

`numpy` is installed in the Python environment at NERSC (`module load python`), but `xarray` is not, so the user must install the package on their own. `PIPPATH` is the assumed location. This notebook depends heavily on `xarray`.

In [ ]:
import os, sys, shutil, glob

# "standard" install
import numpy as np
from tqdm import tqdm

from multiprocessing import Pool

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
# PIPPATH = '/global/homes/e/emlawer/.local/cori/3.8-anaconda-2020.11/' + \
#      'lib/python3.8/site-packages'
PATHS = ['common', PIPPATH]
for path in PATHS: sys.path.append(path)

# needed at AER unless i update `pandas`
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# user must do `pip install xarray` on cori (or other NERSC machines)
import xarray as xa

# local modules
import g_point_reduction as REDUX
import flux_cost_compute as FCC

# Static Inputs

In [ ]:
# only do one domain or the other
DOLW = True
DOMAIN = 'LW' if DOLW else 'SW'
NBANDS = 16 if DOLW else 14

# does band-splitting need to be done, or are there existing files 
# that have divided up the full k-distribution?
BANDSPLIT = True

# Paths

In [ ]:
PROJECT = '/global/project/projectdirs/e3sm/pernak18/'
EXE = '{}/g-point-reduction/garand_atmos/rrtmgp_garand_atmos'.format(
    PROJECT)
REFDIR = '{}/reference_netCDF/g-point-reduce'.format(PROJECT)

# test (RRTMGP) and reference (LBL) flux netCDF files, full k-distributions, 
# and by-band Garand input file
fluxSuffix = 'flux-inputs-outputs-garandANDpreind.nc'
if DOLW:
    GARAND = '{}/multi_garand_template_single_band.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-lw-g256-2018-12-04.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-lw-g256-jen-xs.nc'.format(REFDIR)
    REFNC = '{}/lblrtm-lw-{}'.format(REFDIR, fluxSuffix)
    TESTNC = '{}/rrtmgp-lw-{}'.format(REFDIR, fluxSuffix)
    #TESTNC = 'rrtmgp-lw-flux-inputs-outputs-garand-all.nc'
else:
    GARAND = '{}/charts_multi_garand_template_single_band.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-sw-g224-2018-12-04.nc'.format(REFDIR)
    REFNC = '{}/charts-sw-{}'.format(REFDIR, fluxSuffix)
    TESTNC = '{}/rrtmgp-sw-{}'.format(REFDIR, fluxSuffix)
# endif LW

BANDSPLITDIR = 'band_k_dist'
FULLBANDFLUXDIR = 'full_band_flux'

for PATH in PATHS: FCC.pathCheck(PATH)

CWD = os.getcwd()

KPICKLE = '{}_k-dist.pickle'.format(DOMAIN)
pickleCost = '{}_cost-optimize.pickle'.format(DOMAIN)

# Band Splitting

Break up full _k_-distribution file into separate distributions for each band, then calculate the corresponding fluxes. This should only need to be run once.

After some clarifications from Robert (30-Nov-2020), I believe the plan of action is:

1. create Nbands k-distribution files
2. drive the Fortran executable Nbands times to produce Nbands flux results
3. the trial g-point combinations then loop over bands and the possible g-point combinations within each band, creating k-distribution and band-wise flux files for each possible combination
4. The Python code assembles broadband fluxes from the band-wise flux files in order to compute the cost functions

In [ ]:
if BANDSPLIT:
    print('Band splitting commenced')
    FCC.pathCheck(BANDSPLITDIR, mkdir=True)
    FCC.pathCheck(FULLBANDFLUXDIR, mkdir=True)
    kFiles, fullBandFluxes = [], []
    for iBand in tqdm(range(NBANDS)):
        # divide full k-distribution into subsets for each band
        kObj = REDUX.gCombine_kDist(KFULLNC, iBand, DOLW, 1, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR, 
            profilesNC=GARAND)
        kFiles.append(kObj.kBandNC)
        kObj.kDistBand()

        # quick, non-parallelized flux calculations (because the 
        # executable is run in one directory)
        FCC.fluxCompute(kObj.kBandNC, kObj.profiles, kObj.exe, 
                           kObj.fullBandFluxDir, kObj.fluxBandNC)
        fullBandFluxes.append(kObj.fluxBandNC)
    # end band loop
    print('Band splitting completed')
else:
    kFiles = sorted(glob.glob('{}/coefficients_{}_band??.nc'.format(
        BANDSPLITDIR, DOMAIN)))
    fullBandFluxes = sorted(glob.glob('{}/flux_{}_band??.nc'.format(
        FULLBANDFLUXDIR, DOMAIN)))

    if len(kFiles) == 0 or len(fullBandFluxes) == 0:
        print('WARNING: set `BANDSPLIT` to `True` and run this cell again')
# endif BANDSPLIT


# Pressure Levels for Cost Function

Pressure levels [Pa] for the Garand atmospheres are printed to standard output with indices that can be used in the cost function:

In [ ]:
with xa.open_dataset(REFNC) as refDS:
    pLev = refDS['p_lev'].isel(record=0)
for iLev, pLev in enumerate(pLev.isel(col=0).values): print(iLev, pLev)

# _g_-Point Combining

Combine _g_-point reduced for bands with full-band fluxes from other bands, find optimal _g_-point combination for given iteration, proceed to next iteration.

First, find all _g_-point combinations for each band. Store the band object in a dictionary for use in flux computation. This cell only needs to be run once, and to save time in development, the dictionary is saved in a `pickle` file and can be loaded in the next cell.

In [ ]:
# this should be parallelized; also is part of preprocessing so we 
# shouldn't have to run it multiple times
kBandDict = {}
for iBand, kFile in tqdm(enumerate(kFiles)):
    #if iBand != 0: continue
    band = iBand + 1
    kObj = REDUX.gCombine_kDist(kFile, iBand, DOLW, 1, 
        fullBandKDir=BANDSPLITDIR, 
        fullBandFluxDir=FULLBANDFLUXDIR)
    kObj.gPointCombine()
    kBandDict['band{:02d}'.format(band)] = kObj

    print('Band {} complete'.format(band))
# end kFile loop

import pickle
with open(KPICKLE, 'wb') as fp: pickle.dump(kBandDict, fp)

Now compute fluxes in parallel for every _g_-point combination -- merging occurs in each band, and these combinations in a given band are used with broadband fluxes from other bands. These concatenations each have an associated `xarray` dataset assigned to it. Cost function components are then calculated based for each dataset, and the one that minimizes the error in the cost function will have its associated netCDF saved to disk.

Uncomment pickling block to restore dictionary from previous cell.

# Reduction and Optimization

Test and reference netCDF files have flux and heating rate arrays of dimension `record` x `col` x `lay`/`lev` and `band` if the array is broken down by band. `record` represents atmospheric specifications that can be used in [forcing scenarios](https://github.com/pernak18/g-point-reduction/wiki/LW-Forcing-Number-Convention#g-point-reduction-convention-).

Alternatively, the atmospheric specifications from any scenario can also be used. "Bare" parameters like `heating_rate` and `flux_net` will be treated as PD specifications, so the user will have to specify explicitly if they want the fluxes or heating rates from other scenarios by using the `flux_*_N` and `heating_rate_N` convention, where `N` is the scenario index as listed in the above list. The same convention applies to band fluxes and HRs. `N` = 0 will work just like `heating_rate` and `flux_net`.

Forcing for this exercise is defined as PI subtracted from scenario (2-6). The convention for these quantities is `*_forcing_N`, where `*` is the typical flux or heating rate (band or broadband) string, and `N` again is the forcing scenario (`N` of 2 would be forcing due to doubling methane).

In [ ]:
# pickling for developement purposes so this dictionary doesn't need 
# to be regenerated for every code change.
import pathlib as PL
import copy
import pickle
with open(KPICKLE, 'rb') as fp: kBandDict = pickle.load(fp)

# components used in cost function computation
# variable names in RRTMGP and LBL flux netCDF file, except for 
# forcing, which has to be specifed with "_forcing" appended to 
# the appropriate array. e.g., "flux_net_forcing" for net flux forcing
# netCDF arrays ('heating_rate', 'flux_net', 'band_flux_net', etc.)
# or forcing scenarios: convention is  ('flux_net_forcing_3') for 
#CFCOMPS = ['flux_dif_net', 'flux_dir_dn', 'heating_rate']
CFCOMPS = ['flux_net','band_flux_net','heating_rate','heating_rate_7',
           'flux_net_forcing_5','flux_net_forcing_6','flux_net_forcing_7',
           'flux_net_forcing_9','flux_net_forcing_10','flux_net_forcing_11',
           'flux_net_forcing_12','flux_net_forcing_13','flux_net_forcing_14',
           'flux_net_forcing_15','flux_net_forcing_16','flux_net_forcing_17',
           'flux_net_forcing_18']
# level indices for each component 
# (e.g., 0 for surface, 41 for Garand TOA)
# one dictionary key per component so each component
# can have its own set of level indices
CFLEVS = {}
CFLEVS['flux_net'] = [0, 26, 42]
CFLEVS['band_flux_net'] = [42]
CFLEVS['heating_rate'] = range(42)
CFLEVS['heating_rate_7'] = range(42)
CFLEVS['flux_net_forcing_5'] = [0, 26, 42]
CFLEVS['flux_net_forcing_6'] = [0, 26, 42]
CFLEVS['flux_net_forcing_7'] = [0, 26, 42]
CFLEVS['flux_net_forcing_9'] = [0, 26, 42]
CFLEVS['flux_net_forcing_10'] = [0, 26, 42]
CFLEVS['flux_net_forcing_11'] = [0, 26, 42]
CFLEVS['flux_net_forcing_12'] = [0, 26, 42]
CFLEVS['flux_net_forcing_13'] = [0, 26, 42]
CFLEVS['flux_net_forcing_14'] = [0, 26, 42]
CFLEVS['flux_net_forcing_15'] = [0, 26, 42]
CFLEVS['flux_net_forcing_16'] = [0, 26, 42]
CFLEVS['flux_net_forcing_17'] = [0, 26, 42]
CFLEVS['flux_net_forcing_18'] = [0, 26, 42]

# weights for each cost function component
CFWGT = [0.6, 0.04, 0.12, 0.12,
         0.01, 0.02, 0.04,
        0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 
        0.005, 0.005, 0.005,
        0.005]

# directory under which to store k-distribution files that optimize 
# the cost function for each iteration and diagnistics (if necessary)
CFDIR = 'fullCF_top-layer_redo_abs_parabola'
FCC.pathCheck(CFDIR, mkdir=True)

# write diagnostic netCDFs with cost function components
DIAGNOSTICS = True

RESTORE = False

if RESTORE:
    assert os.path.exists(pickleCost), 'Cannot find {}'.format(pickleCost)
    print('Restoring {}'.format(pickleCost))
    with open(pickleCost, 'rb') as fp: coObj = pickle.load(fp)
else:
    # instantiate object for computing cost
    coObj = REDUX.gCombine_Cost(
        kBandDict, fullBandFluxes, REFNC, TESTNC, 1, 
        DOLW, profilesNC=GARAND, exeRRTMGP=EXE, 
        costFuncComp=CFCOMPS, costFuncLevs=CFLEVS, 
        costWeights=CFWGT, optDir='./{}'.format(CFDIR))
# endif RESTORE

# number of iterations for the optimization
NITER = 1

for i in range(coObj.iCombine, NITER+1):
    wgtInfo = ['{:.2f} ({})'.format(
        wgt, comp) for wgt, comp in zip(CFWGT, CFCOMPS)]
    wgtInfo = ' '.join(wgtInfo)

    print('Iteration {}'.format(i))
    coObj.kMap()
    coObj.fluxComputePool()
    coObj.fluxCombine()
    coObj.costFuncComp(init=True)
    coObj.costFuncComp()
    coObj.findOptimal()
    if coObj.optimized: break
    if DIAGNOSTICS: coObj.costDiagnostics()

    # Karen's additions to `flux_compute.py` for parabola

    import copy

 # Start of special g-point combination branch
    if coObj.dCost[coObj.iOpt]-coObj.deltaCost0 > 0.1:
    #if coObj.dCost[coObj.iOpt]-coObj.deltaCost0 > -2.01:
       print ('will change here')
       xWeight = 0.05
       delta0 = coObj.dCost[coObj.iOpt]-coObj.deltaCost0
       bandObj = coObj.distBands
       if (coObj.optBand+1) < 10:
           bandKey='band0{}'.format(coObj.optBand+1)
       else:
           bandKey='band{}'.format(coObj.optBand+1)
       #sys.exit() 
        
       newObj = REDUX.gCombine_kDist(bandObj[bandKey].kInNC, coObj.optBand, DOLW,
            i, fullBandKDir=BANDSPLITDIR,
            fullBandFluxDir=FULLBANDFLUXDIR)
       curkFile = os.path.basename(coObj.optNC)
       ind = curkFile.find('_g')
       g1 = int(curkFile[ind+2:ind+4])
       g2 = int(curkFile[ind+5:ind+7])
       gCombine =[[g1-1,g2-1]]

       #print (newObj.workDir)
       ind = curkFile.find('coeff')
       #print (curkFile)
       fluxPath = PL.Path(curkFile[ind:]).with_suffix('')
       fluxDir = '{}/{}'.format(newObj.workDir,fluxPath)

       parr =['plus','2plus']
       for pmFlag in parr:
           coCopy = copy.deepcopy(coObj)
           #print ("  ")
           print (pmFlag)
           newObj.gPointCombineSglPair(pmFlag,gCombine,xWeight)
           newCoefFile = '{}/{}_{}.nc'.format(newObj.workDir,fluxPath,pmFlag)
           fluxFile = os.path.basename(newCoefFile).replace('coefficients', 'flux')
           #print (fluxFile)
           #print (newCoefFile)
           FCC.fluxCompute(newCoefFile,GARAND,EXE,fluxDir,fluxFile)

           trialNC = '{}/{}'.format(fluxDir,fluxFile)
           coCopy.combinedNC[coObj.iOpt] = combineBandsSgl( 
                   coObj.optBand, coObj.fullBandFluxes,trialNC,DOLW)
           coCopy.costFuncCompSgl(coCopy.combinedNC[coObj.iOpt])
           #coCopy.findOptimal()
        
           if DIAGNOSTICS: coCopy.costDiagnostics()
           if(pmFlag == '2plus'):
               delta2Plus = coCopy.dCost[coObj.iOpt]-coCopy.deltaCost0
           if(pmFlag == 'plus'):
               deltaPlus = coCopy.dCost[coObj.iOpt]-coCopy.deltaCost0

# Fit change in cost of three g-point options to a parabola and find minimum weight if parabola is concave;
#  if that weight is outside the (-0.1,0.1) range or the parabola is convex use the weight that leads to 
# the smallest increase or largest decrease in the  cost function;

       #print ("  ")
       print ("New weight calculation")
       #xArr = [-xWeight,0.,xWeight]
       xArr = [0.,xWeight,2.*xWeight]
       #yArr = [deltaMinus,delta0,deltaPlus]
       yArr = [delta0,deltaPlus,delta2Plus]
       print (yArr)
       ymin = min(yArr)
       imin = yArr.index(ymin)
       coeff = np.polyfit(xArr,yArr,2)
       #print (coeff[0])
       xMin = -coeff[1]/(2.*coeff[0])
       if (yArr[imin] < 0):
          xWeightNew = xArr[imin] - xArr[imin] * yArr[imin] / (yArr[imin]-yArr[0])
       else:       
          if (coeff[0] >0.):
             print ("concave parabola ",xMin)
             if (xMin < -xWeight or xMin > xWeight):
                xWeightNew = xArr[imin]
             else:
                xWeightNew = xMin
          else:
             xWeightNew = xArr[imin]
       print (xWeightNew)

# Define newest g-point combination
       coCopy = copy.deepcopy(coObj)
       pmFlag = 'mod'
       #print ("  ")
       print (pmFlag)
       newObj.gPointCombineSglPair(pmFlag,gCombine,xWeightNew)
       newCoefFile = '{}/{}_{}.nc'.format(newObj.workDir,fluxPath,pmFlag)
       fluxFile = os.path.basename(newCoefFile).replace('coefficients', 'flux')
       #print (newCoefFile)
       print (newCoefFile,file=open('new_weight_diag.txt','a'))
       print (coeff[0],xMin,yArr,xWeightNew,file=open('new_weight_diag.txt','a'))
       print ("  ",file=open('new_weight_diag.txt','a'))
       FCC.fluxCompute(newCoefFile,GARAND,EXE,fluxDir,fluxFile)

       trialNC = '{}/{}'.format(fluxDir,fluxFile)
       coCopy.combinedNC[coObj.iOpt] = combineBandsSgl( 
               coObj.optBand, coObj.fullBandFluxes,trialNC,DOLW,)
       coCopy.costFuncCompSgl(coCopy.combinedNC[coObj.iOpt])
       coCopy.fluxInputsAll[coObj.iOpt]['fluxNC'] = str(trialNC)
       #coCopy.findOptimal()
    
       if DIAGNOSTICS: coCopy.costDiagnostics()
       print ("delta cost")
       print(coCopy.dCost[coObj.iOpt]-coCopy.deltaCost0)
       coObj = copy.deepcopy(coCopy)
       del coCopy
# End of special g-point combination branch

    # end of Karen's additions to `flux_compute.py` for parabola

    coObj.setupNextIter()
    with open(pickleCost, 'wb') as fp: pickle.dump(coObj, fp)
    coObj.calcOptFlux(
        fluxOutNC='optimized_{}_fluxes_iter{:03d}.nc'.format(DOMAIN, i))
# end iteration loop

KOUTNC = 'rrtmgp-data-{}-g-red.nc'.format(DOMAIN)
coObj.kDistOpt(KFULLNC, kOutNC=KOUTNC)
coObj.calcOptFlux(fluxOutNC='optimized_{}_fluxes.nc'.format(DOMAIN))